In [ ]:
"""
BSD 3-Clause License

Copyright (c) 2019, HJ Reachability Group
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

Author(s): David Fridovich-Keil ( dfk@eecs.berkeley.edu )
"""
################################################################################
#
# Script to run a 3 player collision avoidance example intended to model
# a T-intersection.
#
################################################################################

import os
import numpy as np
import matplotlib.pyplot as plt

from unicycle_4d import Unicycle4D
from bicycle_4d import Bicycle4D
from car_5d import Car5D
from product_multiplayer_dynamical_system import \
    ProductMultiPlayerDynamicalSystem

from point import Point
from polyline import Polyline

from ilq_solver import ILQSolver
from proximity_cost import ProximityCost
from product_state_proximity_cost import ProductStateProximityCost
from semiquadratic_cost import SemiquadraticCost
from quadratic_cost import QuadraticCost
from semiquadratic_polyline_cost import SemiquadraticPolylineCost
from quadratic_polyline_cost import QuadraticPolylineCost
from player_cost import PlayerCost
from box_constraint import BoxConstraint
from nominal_velocity_deviation_cost import NominalVelocityDeviationCost

from visualizer import Visualizer
from logger import Logger

# General parameters.
TIME_HORIZON = 5.0    # s
TIME_RESOLUTION = 0.1 # s
HORIZON_STEPS = int(TIME_HORIZON / TIME_RESOLUTION)
LOG_DIRECTORY = "./logs/three_player/"

# Create dynamics.
car1 = Car5D(4.0)
car2 = Car5D(4.0)
unicycle = Unicycle4D()
dynamics = ProductMultiPlayerDynamicalSystem(
    [car1, car2, unicycle], T=TIME_RESOLUTION)

# Choose initial states and set initial control laws to zero, such that
# we start with a situation that looks like this:
#
#              (car 2)
#             |   X   .       |
#             |   :   .       |
#             |  \./  .       |
# (unicycle) X-->     .       |
#             |       .        ------------------
#             |       .
#             |       .        ..................
#             |       .
#             |       .        ------------------
#             |       .   ^   |
#             |       .   :   |         (+y)
#             |       .   :   |          |
#             |       .   X   |          |
#                      (car 1)           |______ (+x)
#
# We shall set up the costs so that car 2 wants to turn and car 1 / unicycle 1
# continue straight in their initial direction of motion.
# We shall assume that lanes are 4 m wide and set the origin to be in the
# bottom left along the road boundary.
car1_theta0 = np.pi / 2.0 # 90 degree heading
car1_v0 = 0.1             # 5 m/s initial speed
car1_x0 = np.array([
    [6.5],
    [0.0],
    [car1_theta0],
    [0.0],
    [car1_v0]
])

car2_theta0 = -np.pi / 2.0 # -90 degree heading
car2_v0 = 10.0              # 2 m/s initial speed
car2_x0 = np.array([
    [1.5],
    [30.0],
    [car2_theta0],
    [0.0],
    [car2_v0]
])

unicycle_theta0 = 1.0 # moving right
unicycle_v0 = 2.0   # 0.1 m/s initial speed
unicycle_x0 = np.array([
    [0.0],
    [17.0],
    [unicycle_theta0],
    [unicycle_v0]
])

stacked_x0 = np.concatenate([car1_x0, car2_x0, unicycle_x0], axis=0)

car1_Ps = [np.zeros((car1._u_dim, dynamics._x_dim))] * HORIZON_STEPS
car2_Ps = [np.zeros((car2._u_dim, dynamics._x_dim))] * HORIZON_STEPS
unicycle_Ps = [np.zeros((unicycle._u_dim, dynamics._x_dim))] * HORIZON_STEPS

car1_alphas = [np.zeros((car1._u_dim, 1))] * HORIZON_STEPS
car2_alphas = [np.zeros((car2._u_dim, 1))] * HORIZON_STEPS
unicycle_alphas = [np.zeros((unicycle._u_dim, 1))] * HORIZON_STEPS

# Create environment:
car1_position_indices_in_product_state = (0, 1)
car1_polyline = Polyline([Point(6.0, -100.0), Point(6.0, 100.0)])
car1_polyline_boundary_cost = SemiquadraticPolylineCost(
    car1_polyline, 1.0, car1_position_indices_in_product_state,
    "car1_polyline_boundary")
car1_polyline_cost = QuadraticPolylineCost(
    car1_polyline, car1_position_indices_in_product_state, "car1_polyline")

car1_goal = Point(6.0, 30.0)
car1_goal_cost = ProximityCost(
    car1_position_indices_in_product_state, car1_goal, np.inf, "car1_goal")

# Environment for Car 2
car2_position_indices_in_product_state = (5, 6)
car2_polyline = Polyline([Point(2.0, 100.0),
                          Point(2.0, 18.0),
                          Point(2.5, 15.0),
                          Point(3.0, 14.0),
                          Point(5.0, 12.5),
                          Point(8.0, 12.0),
                          Point(100.0, 12.0)])
car2_polyline_boundary_cost = SemiquadraticPolylineCost(
    car2_polyline, 1.0, car2_position_indices_in_product_state,
    "car2_polyline_boundary")
car2_polyline_cost = QuadraticPolylineCost(
    car2_polyline, car2_position_indices_in_product_state, "car2_polyline")

car2_goal = Point(20.0, 12.0)
car2_goal_cost = ProximityCost(
    car2_position_indices_in_product_state, car2_goal, np.inf, "car2_goal")


# Environment for Unicycle
unicycle_position_indices_in_product_state = (10, 11)
unicycle_polyline = Polyline([Point(0.0, 17.0),
                          Point(18.0, 17.0)])
unicycle_polyline_boundary_cost = SemiquadraticPolylineCost(
    unicycle_polyline, 1.0, unicycle_position_indices_in_product_state,
    "unicycle_polyline_boundary")
unicycle_polyline_cost = QuadraticPolylineCost(
    unicycle_polyline, unicycle_position_indices_in_product_state, "unicycle_polyline")


unicycle_goal = Point(15.0, 17.0)
unicycle_goal_cost = ProximityCost(
    unicycle_position_indices_in_product_state, unicycle_goal, np.inf, "unicycle_goal")

# Penalize speed above a threshold for all players.
car1_v_index_in_product_state = 4
car1_maxv = 8.0 # m/s
car1_minv_cost = SemiquadraticCost(
    car1_v_index_in_product_state, 0.0, False, "car1_minv")
car1_maxv_cost = SemiquadraticCost(
    car1_v_index_in_product_state, car1_maxv, True, "car1_maxv")

car2_v_index_in_product_state = 9
car2_maxv = 8.0 # m/s
car2_minv_cost = SemiquadraticCost(
    car2_v_index_in_product_state, 0.0, False, "car2_minv")
car2_maxv_cost = SemiquadraticCost(
    car2_v_index_in_product_state, car2_maxv, True, "car2_maxv")

unicycle_v_index_in_product_state = 13
unicycle_maxv = 2.0 # m/s
unicycle_minv_cost = SemiquadraticCost(
    unicycle_v_index_in_product_state, 0.0, False, "unicycle_minv")
unicycle_maxv_cost = SemiquadraticCost(
    unicycle_v_index_in_product_state, unicycle_maxv, True, "unicycle_maxv")


# Penalize deviation from nominal speed for all players
car1_nominalv = 0.0 # 8.0
car1_nom_vel_dev = NominalVelocityDeviationCost(car1_v_index_in_product_state, car1_nominalv, "car1_nom_v_dev")

car2_nominalv = 0.0 # 6.0
car2_nom_vel_dev = NominalVelocityDeviationCost(car2_v_index_in_product_state, car1_nominalv, "car2_nom_v_dev")

unicycle_nominalv = 0.0 # 1.0
unicycle_nom_vel_dev = NominalVelocityDeviationCost(car2_v_index_in_product_state, car1_nominalv, "unicycle_nom_v_dev")


# Control costs for all players.
car1_steering_cost = QuadraticCost(0, 0.0, "car1_steering")
car1_a_cost = QuadraticCost(1, 0.0, "car1_a")

car2_steering_cost = QuadraticCost(0, 0.0, "car2_steering")
car2_a_cost = QuadraticCost(1, 0.0, "car2_a")

unicycle_steering_cost = QuadraticCost(0, 0.0, "unicycle_steering")
unicycle_a_cost = QuadraticCost(1, 0.0, "unicycle_a")

# Player ids
car1_player_id = 0
car2_player_id = 1
unicycle_player_id = 2

# Proximity cost.
CAR_PROXIMITY_THRESHOLD = 3.0
UNICYCLE_PROXIMITY_THRESHOLD = 1.0
car1_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state,
     unicycle_position_indices_in_product_state],
    CAR_PROXIMITY_THRESHOLD,
    car1_player_id,
    "car1_proximity")
car2_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state,
     unicycle_position_indices_in_product_state],
    CAR_PROXIMITY_THRESHOLD,
    car2_player_id,
    "car2_proximity")
unicycle_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state,
     unicycle_position_indices_in_product_state],
    UNICYCLE_PROXIMITY_THRESHOLD,
    unicycle_player_id,
    "unicycle_proximity")

# Build up total costs for both players. This is basically a zero-sum game.
car1_cost = PlayerCost()
car1_cost.add_cost(car1_goal_cost, "x", 30.0) # -1.0
car1_cost.add_cost(car1_polyline_cost, "x", 50.0) # 50.0
car1_cost.add_cost(car1_polyline_boundary_cost, "x", 200.0) # 200.0
car1_cost.add_cost(car1_maxv_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_minv_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_proximity_cost, "x", 300.0) # 100.0
car1_cost.add_cost(car1_nom_vel_dev, "x", 50.0)

car1_player_id = 0
car1_cost.add_cost(car1_steering_cost, car1_player_id, 50.0) # 50.0
car1_cost.add_cost(car1_a_cost, car1_player_id, 1.0) # 1.0

car2_cost = PlayerCost()
car2_cost.add_cost(car2_goal_cost, "x", 30.0) # -1.0
car2_cost.add_cost(car2_polyline_cost, "x", 250.0) #50.0
car2_cost.add_cost(car2_polyline_boundary_cost, "x", 100.0) # 200.0
car2_cost.add_cost(car2_maxv_cost, "x", 80.0) # 100.0
car2_cost.add_cost(car2_minv_cost, "x", 80.0) # 100.0
car2_cost.add_cost(car2_proximity_cost, "x", 100.0) # 100.0
car2_cost.add_cost(car2_nom_vel_dev, "x", 50.0)

car2_player_id = 1
car2_cost.add_cost(car2_steering_cost, car2_player_id, 50.0) # 50.0
car2_cost.add_cost(car2_a_cost, car2_player_id, 1.0) # 1.0

unicycle_cost = PlayerCost()
unicycle_cost.add_cost(unicycle_goal_cost, "x", 30.0) # -1.0
unicycle_cost.add_cost(unicycle_maxv_cost, "x", 50.0) # 100.0
unicycle_cost.add_cost(unicycle_minv_cost, "x", 50.0) # 100.0
unicycle_cost.add_cost(unicycle_proximity_cost, "x", 5.0) # 5.0
unicycle_cost.add_cost(unicycle_nom_vel_dev, "x", 50.0)
unicycle_cost.add_cost(unicycle_polyline_cost, "x", 100.0) #50.0
unicycle_cost.add_cost(unicycle_polyline_boundary_cost, "x", 50.0) # 200.0

unicycle_player_id = 2
unicycle_cost.add_cost(unicycle_steering_cost, unicycle_player_id, 50.0) # 50.0
unicycle_cost.add_cost(unicycle_a_cost, unicycle_player_id, 25.0) # 1.0

# Visualizer.
visualizer = Visualizer(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state,
     unicycle_position_indices_in_product_state],
    [car1_polyline_boundary_cost,
     car1_goal_cost,
     car2_polyline_boundary_cost,
     car2_goal_cost,
     unicycle_goal_cost],
    [".-r", ".-g", ".-b"],
    1,
    False,
    plot_lims=[-5, 25, -5, 35])

# Logger.
if not os.path.exists(LOG_DIRECTORY):
    os.makedirs(LOG_DIRECTORY)

logger = Logger(os.path.join(LOG_DIRECTORY, 'intersection_car_example.pkl'))

# Set up ILQSolver.
solver = ILQSolver(dynamics,
                   [car1_cost, car2_cost, unicycle_cost],
                   stacked_x0,
                   [car1_Ps, car2_Ps, unicycle_Ps],
                   [car1_alphas, car2_alphas, unicycle_alphas],
                   0.1,
                   None,
                   logger,
                   visualizer,
                   None)

solver.run()

In [ ]:
"""
BSD 3-Clause License

Copyright (c) 2019, HJ Reachability Group
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

Author(s): David Fridovich-Keil ( dfk@eecs.berkeley.edu )
"""
################################################################################
#
# Script to run a 3 player collision avoidance example intended to model
# a T-intersection.
#
################################################################################

import os
import numpy as np
import matplotlib.pyplot as plt

from unicycle_4d import Unicycle4D
from bicycle_4d import Bicycle4D
from car_5d import Car5D
from product_multiplayer_dynamical_system import \
    ProductMultiPlayerDynamicalSystem

from point import Point
from polyline import Polyline

from ilq_solver import ILQSolver
from proximity_cost import ProximityCost
from product_state_proximity_cost import ProductStateProximityCost
from semiquadratic_cost import SemiquadraticCost
from quadratic_cost import QuadraticCost
from semiquadratic_polyline_cost import SemiquadraticPolylineCost
from quadratic_polyline_cost import QuadraticPolylineCost
from player_cost import PlayerCost
from box_constraint import BoxConstraint
from nominal_velocity_deviation_cost import NominalVelocityDeviationCost

from visualizer import Visualizer
from logger import Logger

# General parameters.
TIME_HORIZON = 5.0    # s
TIME_RESOLUTION = 0.1 # s
HORIZON_STEPS = int(TIME_HORIZON / TIME_RESOLUTION)
LOG_DIRECTORY = "./logs/three_player/"

# Create dynamics.
car1 = Car5D(4.0)
car2 = Car5D(4.0)
dynamics = ProductMultiPlayerDynamicalSystem(
    [car1, car2], T=TIME_RESOLUTION)

# Choose initial states and set initial control laws to zero, such that
# we start with a situation that looks like this:
#
#              (car 2)
#             |   X   .       |
#             |   :   .       |
#             |  \./  .       |
# (unicycle) X-->     .       |
#             |       .        ------------------
#             |       .
#             |       .        ..................
#             |       .
#             |       .        ------------------
#             |       .   ^   |
#             |       .   :   |         (+y)
#             |       .   :   |          |
#             |       .   X   |          |
#                      (car 1)           |______ (+x)
#
# We shall set up the costs so that car 2 wants to turn and car 1 / unicycle 1
# continue straight in their initial direction of motion.
# We shall assume that lanes are 4 m wide and set the origin to be in the
# bottom left along the road boundary.
car1_theta0 = np.pi / 2.0 # 90 degree heading
car1_v0 = 0.1             # 5 m/s initial speed
car1_x0 = np.array([
    [6.5],
    [0.0],
    [car1_theta0],
    [0.0],
    [car1_v0]
])

car2_theta0 = -np.pi / 2.0 # -90 degree heading
car2_v0 = 10.0              # 2 m/s initial speed
car2_x0 = np.array([
    [1.5],
    [30.0],
    [car2_theta0],
    [0.0],
    [car2_v0]
])


stacked_x0 = np.concatenate([car1_x0, car2_x0], axis=0)

car1_Ps = [np.zeros((car1._u_dim, dynamics._x_dim))] * HORIZON_STEPS
car2_Ps = [np.zeros((car2._u_dim, dynamics._x_dim))] * HORIZON_STEPS

car1_alphas = [np.zeros((car1._u_dim, 1))] * HORIZON_STEPS
car2_alphas = [np.zeros((car2._u_dim, 1))] * HORIZON_STEPS


# Create environment.
car1_position_indices_in_product_state = (0, 1)
car1_polyline = Polyline([Point(6.0, -100.0), Point(6.0, 100.0)])
car1_polyline_boundary_cost = SemiquadraticPolylineCost(
    car1_polyline, 1.0, car1_position_indices_in_product_state,
    "car1_polyline_boundary")
car1_polyline_cost = QuadraticPolylineCost(
    car1_polyline, car1_position_indices_in_product_state, "car1_polyline")

car1_goal = Point(6.0, 30.0)
car1_goal_cost = ProximityCost(
    car1_position_indices_in_product_state, car1_goal, np.inf, "car1_goal")

car2_position_indices_in_product_state = (5, 6)
car2_polyline = Polyline([Point(2.0, 100.0),
                          Point(2.0, 18.0),
                          Point(2.5, 15.0),
                          Point(3.0, 14.0),
                          Point(5.0, 12.5),
                          Point(8.0, 12.0),
                          Point(100.0, 12.0)])
car2_polyline_boundary_cost = SemiquadraticPolylineCost(
    car2_polyline, 1.0, car2_position_indices_in_product_state,
    "car2_polyline_boundary")
car2_polyline_cost = QuadraticPolylineCost(
    car2_polyline, car2_position_indices_in_product_state, "car2_polyline")

car2_goal = Point(20.0, 12.0)
car2_goal_cost = ProximityCost(
    car2_position_indices_in_product_state, car2_goal, np.inf, "car2_goal")


# Penalize speed above a threshold for all players.
car1_v_index_in_product_state = 4
car1_maxv = 15.0 # m/s
car1_minv_cost = SemiquadraticCost(
    car1_v_index_in_product_state, 0.0, False, "car1_minv")
car1_maxv_cost = SemiquadraticCost(
    car1_v_index_in_product_state, car1_maxv, True, "car1_maxv")

car2_v_index_in_product_state = 9
car2_maxv = 15.0 # m/s
car2_minv_cost = SemiquadraticCost(
    car2_v_index_in_product_state, 0.0, False, "car2_minv")
car2_maxv_cost = SemiquadraticCost(
    car2_v_index_in_product_state, car2_maxv, True, "car2_maxv")


# Penalize deviation from nominal speed for all players
car1_nominalv = 8.0
car1_nom_vel_dev = NominalVelocityDeviationCost(car1_v_index_in_product_state, car1_nominalv, "car1_nom_v_dev")

car2_nominalv = 6.0
car2_nom_vel_dev = NominalVelocityDeviationCost(car2_v_index_in_product_state, car1_nominalv, "car2_nom_v_dev")


# Control costs for all players.
car1_steering_cost = QuadraticCost(0, 0.0, "car1_steering")
car1_a_cost = QuadraticCost(1, 0.0, "car1_a")

car2_steering_cost = QuadraticCost(0, 0.0, "car2_steering")
car2_a_cost = QuadraticCost(1, 0.0, "car2_a")


# Player ids
car1_player_id = 0
car2_player_id = 1

# Proximity cost.
CAR_PROXIMITY_THRESHOLD = 3.0
car1_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state],
    CAR_PROXIMITY_THRESHOLD,
    car1_player_id,
    "car1_proximity")
car2_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state],
    CAR_PROXIMITY_THRESHOLD,
    car2_player_id,
    "car2_proximity")


# Build up total costs for both players. This is basically a zero-sum game.
car1_cost = PlayerCost()
car1_cost.add_cost(car1_goal_cost, "x", 10.0) # -1.0
car1_cost.add_cost(car1_polyline_cost, "x", 50.0) # 50.0
car1_cost.add_cost(car1_polyline_boundary_cost, "x", 200.0) # 200.0
car1_cost.add_cost(car1_maxv_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_minv_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_proximity_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_nom_vel_dev, "x", 100.0)

car1_player_id = 0
car1_cost.add_cost(car1_steering_cost, car1_player_id, 50.0) # 50.0
car1_cost.add_cost(car1_a_cost, car1_player_id, 1.0) # 1.0

car2_cost = PlayerCost()
car2_cost.add_cost(car2_goal_cost, "x", 80.0) # -1.0
car2_cost.add_cost(car2_polyline_cost, "x", 50.0) #50.0
car2_cost.add_cost(car2_polyline_boundary_cost, "x", 200.0) # 200.0
car2_cost.add_cost(car2_maxv_cost, "x", 100.0) # 100.0
car2_cost.add_cost(car2_minv_cost, "x", 100.0) # 100.0
car2_cost.add_cost(car2_proximity_cost, "x", 100.0) # 100.0
car2_cost.add_cost(car2_nom_vel_dev, "x", 100.0)

car2_player_id = 1
car2_cost.add_cost(car2_steering_cost, car2_player_id, 50.0) # 50.0
car2_cost.add_cost(car2_a_cost, car2_player_id, 1.0) # 1.0


# Visualizer.
visualizer = Visualizer(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state],
    [car1_polyline_boundary_cost,
     car1_goal_cost,
     car2_polyline_boundary_cost,
     car2_goal_cost],
    [".-r", ".-g", ".-b"],
    1,
    False,
    plot_lims=[-5, 25, -5, 35])

# Logger.
if not os.path.exists(LOG_DIRECTORY):
    os.makedirs(LOG_DIRECTORY)

logger = Logger(os.path.join(LOG_DIRECTORY, 'intersection_car_example.pkl'))

# Set up ILQSolver.
solver = ILQSolver(dynamics,
                   [car1_cost, car2_cost],
                   stacked_x0,
                   [car1_Ps, car2_Ps],
                   [car1_alphas, car2_alphas],
                   0.1,
                   None,
                   logger,
                   visualizer,
                   None)

solver.run()

In [ ]:
"""
BSD 3-Clause License

Copyright (c) 2019, HJ Reachability Group
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

Author(s): David Fridovich-Keil ( dfk@eecs.berkeley.edu )
"""
################################################################################
#
# Script to run a 3 player collision avoidance example intended to model
# a T-intersection.
#
################################################################################

import os
import numpy as np
import matplotlib.pyplot as plt

from unicycle_4d import Unicycle4D
from bicycle_4d import Bicycle4D
from car_5d import Car5D
from product_multiplayer_dynamical_system import \
    ProductMultiPlayerDynamicalSystem

from point import Point
from polyline import Polyline

from ilq_solver_base import ILQSolver
from proximity_cost import ProximityCost
from product_state_proximity_cost import ProductStateProximityCost
from semiquadratic_cost import SemiquadraticCost
from quadratic_cost import QuadraticCost
from semiquadratic_polyline_cost import SemiquadraticPolylineCost
from quadratic_polyline_cost import QuadraticPolylineCost
from player_cost import PlayerCost
from box_constraint import BoxConstraint
from nominal_velocity_deviation_cost import NominalVelocityDeviationCost

from visualizer import Visualizer
from logger import Logger

# General parameters.
TIME_HORIZON = 3.0    # s
TIME_RESOLUTION = 0.1 # s
HORIZON_STEPS = int(TIME_HORIZON / TIME_RESOLUTION)
LOG_DIRECTORY = "./logs/three_player/"

# Create dynamics.
car1 = Car5D(4.0)
car2 = Car5D(4.0)
dynamics = ProductMultiPlayerDynamicalSystem(
    [car1, car2], T=TIME_RESOLUTION)

# Choose initial states and set initial control laws to zero, such that
# we start with a situation that looks like this:
#
#              (car 2)
#             |   X   .       |
#             |   :   .       |
#             |  \./  .       |
# (unicycle) X-->     .       |
#             |       .        ------------------
#             |       .
#             |       .        ..................
#             |       .
#             |       .        ------------------
#             |       .   ^   |
#             |       .   :   |         (+y)
#             |       .   :   |          |
#             |       .   X   |          |
#                      (car 1)           |______ (+x)
#
# We shall set up the costs so that car 2 wants to turn and car 1 / unicycle 1
# continue straight in their initial direction of motion.
# We shall assume that lanes are 4 m wide and set the origin to be in the
# bottom left along the road boundary.
car1_theta0 = np.pi / 2.05 # 90 degree heading
car1_v0 = 0.1             # 5 m/s initial speed
car1_x0 = np.array([
    [6.5],
    [0.0],
    [car1_theta0],
    [0.0],
    [car1_v0]
])

car2_theta0 = -np.pi / 1.95 # -90 degree heading
car2_v0 = 0.1              # 2 m/s initial speed
car2_x0 = np.array([
    [6.5],
    [30.0],
    [car2_theta0],
    [0.0],
    [car2_v0]
])


stacked_x0 = np.concatenate([car1_x0, car2_x0], axis=0)

car1_Ps = [np.zeros((car1._u_dim, dynamics._x_dim))] * HORIZON_STEPS
car2_Ps = [np.zeros((car2._u_dim, dynamics._x_dim))] * HORIZON_STEPS

car1_alphas = [np.zeros((car1._u_dim, 1))] * HORIZON_STEPS
car2_alphas = [np.zeros((car2._u_dim, 1))] * HORIZON_STEPS


# Create environment.
car1_position_indices_in_product_state = (0, 1)
car1_polyline = Polyline([Point(6.0, -100.0), Point(6.0, 100.0)])
car1_polyline_boundary_cost = SemiquadraticPolylineCost(
    car1_polyline, 1.0, car1_position_indices_in_product_state,
    "car1_polyline_boundary")
car1_polyline_cost = QuadraticPolylineCost(
    car1_polyline, car1_position_indices_in_product_state, "car1_polyline")

car1_goal = Point(6.5, 30.0)
car1_goal_cost = ProximityCost(
    car1_position_indices_in_product_state, car1_goal, np.inf, "car1_goal")

car2_position_indices_in_product_state = (5, 6)
car2_polyline = Polyline([Point(6.0, -100.0), Point(6.0, 100.0)])
car2_polyline_boundary_cost = SemiquadraticPolylineCost(
    car2_polyline, 1.0, car2_position_indices_in_product_state,
    "car2_polyline_boundary")
car2_polyline_cost = QuadraticPolylineCost(
    car2_polyline, car2_position_indices_in_product_state, "car2_polyline")

car2_goal = Point(6.5, 0.0)
car2_goal_cost = ProximityCost(
    car2_position_indices_in_product_state, car2_goal, np.inf, "car2_goal")


# Penalize speed above a threshold for all players.
car1_v_index_in_product_state = 4
car1_maxv = 15.0 # m/s
car1_minv_cost = SemiquadraticCost(
    car1_v_index_in_product_state, 0.0, False, "car1_minv")
car1_maxv_cost = SemiquadraticCost(
    car1_v_index_in_product_state, car1_maxv, True, "car1_maxv")

car2_v_index_in_product_state = 9
car2_maxv = 15.0 # m/s
car2_minv_cost = SemiquadraticCost(
    car2_v_index_in_product_state, 0.0, False, "car2_minv")
car2_maxv_cost = SemiquadraticCost(
    car2_v_index_in_product_state, car2_maxv, True, "car2_maxv")


# Penalize deviation from nominal speed for all players
car1_nominalv = 8.0
car1_nom_vel_dev = NominalVelocityDeviationCost(car1_v_index_in_product_state, car1_nominalv, "car1_nom_v_dev")

car2_nominalv = 6.0
car2_nom_vel_dev = NominalVelocityDeviationCost(car2_v_index_in_product_state, car1_nominalv, "car2_nom_v_dev")


# Control costs for all players.
car1_steering_cost = QuadraticCost(0, 0.0, "car1_steering")
car1_a_cost = QuadraticCost(1, 0.0, "car1_a")

car2_steering_cost = QuadraticCost(0, 0.0, "car2_steering")
car2_a_cost = QuadraticCost(1, 0.0, "car2_a")


# Player ids
car1_player_id = 0
car2_player_id = 1

# Proximity cost.
CAR_PROXIMITY_THRESHOLD = 5.0
car1_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state],
    CAR_PROXIMITY_THRESHOLD,
    car1_player_id,
    "car1_proximity")
car2_proximity_cost = ProductStateProximityCost(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state],
    CAR_PROXIMITY_THRESHOLD,
    car2_player_id,
    "car2_proximity")


# Build up total costs for both players. This is basically a zero-sum game.
car1_cost = PlayerCost()
car1_cost.add_cost(car1_goal_cost, "x", 10.0) # -1.0
car1_cost.add_cost(car1_polyline_cost, "x", 50.0) # 50.0
car1_cost.add_cost(car1_polyline_boundary_cost, "x", 200.0) # 200.0
car1_cost.add_cost(car1_maxv_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_minv_cost, "x", 100.0) # 100.0
car1_cost.add_cost(car1_proximity_cost, "x", 10.0) # 100.0
car1_cost.add_cost(car1_nom_vel_dev, "x", 100.0)

car1_player_id = 0
car1_cost.add_cost(car1_steering_cost, car1_player_id, 260.0) # 50.0
car1_cost.add_cost(car1_a_cost, car1_player_id, 70.0) # 1.0

car2_cost = PlayerCost()
car2_cost.add_cost(car2_goal_cost, "x", 150.0) # -1.0
car2_cost.add_cost(car2_polyline_cost, "x", 80.0) #50.0
car2_cost.add_cost(car2_polyline_boundary_cost, "x", 50.0) # 200.0
car2_cost.add_cost(car2_maxv_cost, "x", 220.0) # 100.0
car2_cost.add_cost(car2_minv_cost, "x", 50.0) # 100.0
car2_cost.add_cost(car2_proximity_cost, "x", 550.0) # 100.0
car2_cost.add_cost(car2_nom_vel_dev, "x", 250.0)

car2_player_id = 1
car2_cost.add_cost(car2_steering_cost, car2_player_id, 300.0) # 50.0
car2_cost.add_cost(car2_a_cost, car2_player_id, 200.0) # 1.0


# Visualizer.
visualizer = Visualizer(
    [car1_position_indices_in_product_state,
     car2_position_indices_in_product_state],
    [car1_polyline_boundary_cost,
     car1_goal_cost,
     car2_polyline_boundary_cost,
     car2_goal_cost],
    [".-r", ".-g", ".-b"],
    1,
    False,
    plot_lims=[-5, 25, -5, 35])

# Logger.
if not os.path.exists(LOG_DIRECTORY):
    os.makedirs(LOG_DIRECTORY)

logger = Logger(os.path.join(LOG_DIRECTORY, 'intersection_car_example.pkl'))

# Set up ILQSolver.
solver = ILQSolver(dynamics,
                   [car1_cost, car2_cost],
                   stacked_x0,
                   [car1_Ps, car2_Ps],
                   [car1_alphas, car2_alphas],
                   0.1,
                   None,
                   logger,
                   visualizer,
                   None)

solver.run()


[car1_proximity, car1_goal, car2_prox, car2_cost]
[car1_pos, car1_goal, string, car2_pos]

In [ ]:
"""
This is me trying to write it by myself.
Every cell below this is function for the ILQ solver
"""
import numpy as np

# Defining time horizon, time-steps and number of steps:
time_horizon = 5.0
time_step = 0.1
num_steps = int(time_horizon / time_step)

car1 = Car5D()
car2 = Car5D()

# Define intial state for car 1:
car1_theta0 = np.pi/2
car1_v0 = 2 # m/s
car1_x0 = np.array([
    [6.5],
    [0.0]
    [car1_theta0],
    [0.0],
    [car1_v0]
])

# Define initial state for car 2:
car2_theta0 = -np.pi/2
car2_v0 = 5 # m/s
car2_x0 = np.array([
    [1.5],
    [30.0]
    [car2_theta0],
    [0.0],
    [car2_v0]
])

stack_x0 = np.concatenate([car1_x0, car2_x0])

# Initialize P's and alpha's for car 1 and car 2:
car1_Ps = [np.zeros((car1._u_dim, dynamics._x_dim))] * num_steps
car1_alphas = [np.zeros((car1._u_dim, 1))] * num_steps

car2_Ps = [np.zeros((car2._u_dim, dynamics._x_dim))] * num_steps
car2_alphas = [np.zeros((car2._u_dim, 1))] * num_steps


# Create environment


In [ ]:
# Here, this is the line segment class

class LineSegment(object):
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
    def __len__(self):
        return np.norm(p1 - p2)
    
    def signed_distance_to(self, point):
        """
        Compute distance signed distance between point and either p1 or p2.
        Sign convention is negative to the left, and positive to the right
        """
        #
        relative = point - p1
        
        # Calculate unit direction of line segment
        direction = self.p1 - self.p2
        direction /= np.norm(direction)
        
        # Find signed length of projection and of cross product.
        projection = relative.x * direction.x + relative.y * direction.y
        cross = relative.x * direction.y - direction.x * relative.y
        cross_sign = 1.0 if cross >= 0.0 else -1.0

        if projection < 0.0:
            # Query lies behind this line segment, so closest distance will be
            # from p1.
            return cross_sign * relative.norm()
        elif projection > self.__len__():
            # Closest distance will be to p2.
            return cross_sign * (self.p2 - point).norm()
        else:
            return cross

In [ ]:
# This is the Polyline class

class Polyline(object):
    """
    Compute signed distance from the polyline to the point
    """
    
    def __init__(self, point=[]):
        """
        :parameter point : list of points
        """
        self.points = points
        
    def signed_distance_to(self, point):
        
        best_signed_distance = float("inf")
        
        # Go through the polylines and find out which one the point is closest to
        for i in range(1,len(self.points)):
            segment = LineSegment(self.points[ii-1], self.points[ii])
            signed_distance = segment.signed_distance_to(point)
            
            if abs(signed_distance) < abs(best_signed_distance):
                best_signed_distance = signed_distance
                
        return best_signed_distance

In [ ]:
# This is to penalize lane boundary

class LaneBoundaryCost(Cost): # semiquadratic_polyline_cost.py
    def __init__(self, polyline, distance_threshold, position_indices, name=""):
        """
        This is the class where we
        
        param polyline: piecewise linear path
        type polyline: Polyline
        param distance_threshold: value above which to penalize
        type threshold: float
        """
        
        self.polyline = polyline
        self.distance_threshold = distance_threshold
        self._x_index, self._y_index = position_indices
        
    def __call__(self, x, k=0):
        
        signed_distance = self.polyline.signed_distance_to(Point(x[self._]))  #FINISH
        
        if abs(signed_distance) > distance_threshold:
            return np.norm(distance_threshold - signed_distance)**2
        
        return 0.0

In [ ]:

class LaneDeviationCost(Cost): # quadratic_polyline_cost
    """
    The is the class where we penalize deviation from the polyline
    """
    
    def __init__(self, polyline, position_indices, name=""):
        self.polyline = polyline
        self._x_index, self._y_index = position_indices
        
    def __call__(self, x, k=0):
        signed_distance = self._polyline.signed_distance_to(Point(x[self._x_index, 0], x[self._y_index, 0]))
        
        return signed_distance**2

In [ ]:

class QuadraticCost(Cost):
    """
    This gives us the quadratic cost of either the state, x, or the control, u
    """
    def __init__(self, dimension, name=""):
        self._dimension = dimension
    
    def __call__(self, xu):
        
        return xu[self._dimension, 0]**2

In [ ]:

class SemiquadraticCost(Cost):
    """
    This is for penalizing something going above or falling below a threshold
    """
    
    def __init__(self, dimension, threshold, above_threshold, name=""):
        self._dimension = dimension
        self._threshold = threshold
        self._above_threshold = above_threshold
        
    def __call__(self, xu, k=0):
        if self._above_threshold:
            if xu[self._dimension, 0] > threshold
            return (xu[self._dimension, 0] - self._threshold)**2
        else:
            if xu[self._dimension, 0] > threshold:
                return (xu[self._dimension, 0] - self._threshold)**2
            
        return 0.0

In [ ]:

class StateProximityCost(Cost): # Product_State_Proximity_Cost
    def __init__(self, position_indices, max_distance, name=""):
        self._position_indices = position_indices
        self._max_distance = max_distance
        self._num_players = len(position_indices)
        
    def __call__(self, x, k=0):
        total_cost = 0.0
        
        for ii in range(self._num_players):
            xi_index, yi_index = self._position_indices[ii]
            
            for jj in range(self._num_players):
                if ii == jj:
                    continue
                    
                # Computing relative distance between both players
                xj_index, yj_index = self._position_indices[jj]
                dx = x[xi_index, 0] - x[xj_index, 0]
                dy = x[yi_index, 0] - x[yj_index, 0]
                relative_dist = math.sqrt(dx*dx + dy*dy)
                
                total_cost += min(max_distance - relative_dist, 0)**2
                
    return total_cost

In [ ]:

class GoalProximityCost(Cost): #Proximity_Cost
    def __init__(self, position_indices, point, max_distance, apply_after_time, name=""):
        self._x_index, self._y_index = position_indices
        self._point = point
        self._max_distance = max_distance
        self._apply_after_time = apply_after_time
        
    def __call__(self, x, k=0):
        if k < self._apply_after_time:
            return 0.0
        
        else:
            # Compute relative distance
            dx = x[self._x_index, 0] - self._point.x
            dy = x[self._y_index, 0] - self._point.y
        
            relative_squared_distance = dx*dx + dy*dy
        
        return relative_squared_distance

In [ ]:

class ProductMultiplayerDynamicalSystem(object):
    def __init__(self, subsystems, T=0.1):
        self._subsystems = subsystems
        self._x_dims = [sys._x_dim for sys in subsystems]
        
        x_dim = sum(self._x_dims)
        u_dims = [sys._u_dim for sys in subsystems]
        
    def __call__(self, x, u):
        

In [ ]:

class DynamicalSystem(object):
    def __init__(self, x_dim, u_dim):
        self._x_dim = x_dim
        self._u_dim = u_dim
        
    def integrate(self, x0, u, dt=None):
        
        # FINISH!!!!!!!!!!!!!!!
        
    def linearize(self, x0, u0):
        # FINISH!!!!!!!!!!!!!!

In [ ]:

class PlayerCost(Cost):
    def __init__(self):
        self._costs = []
        self._args = []
        self._weights = []
        
    def __call__(self, x, u, k):
        for cost, arg, weight in zip(self._costs, self._args, self._weights):
            if arg == "x":
                cost_input = x
            else:
                cost_input = u[arg]
                
            current_term = weight * cost(cost_input, k)
            if current_term > 1e7:
                print("Warning: cost %s is %f", %(cost._name, current_term))
                print("Input is: ", cost_input)
                
            if first_time_through:
                total_cost = current_term
            else:
                total_cost += current_term
                
        return total_cost
    
    
    
    
    def quadraticize(self, x, u, k):
        
        
        # Compute gradient of the cost w.r.t. the state x
        grad_x_torch = torch.autograd.grad(cost_torch, x_torch)
        grad_u_torch = [torch.autograd.grad(cost_torch, ui_torch)[0]
        
        
        hess_x = np.zeros((len(x), len(x)))
        grad_x = np.zeros((len(x), 1))
        
                        # FINISH!!!!!!!!!!
        

In [ ]:

class ILQSolver(object):
    def __init__(self, dynamics, player_costs, x0, Ps, alphas, alpha_scaling=0.05, logger=None, visualizer=None, u_constraints=None):
        
        self._dynamics = dynamics
        self._player_costs = player_costs
        self._x0 = x0
        self._Ps = Ps
        self._alphas = alphas
        self._horizon = len(Ps[0])
        self._num_players = len(player_costs)
        
        # Current and previous operating points (states and controls) in order to check if we've converged
        self._last_operating_point = None
        self._current_operating_point = None
        
        # Fix the step size for the linesearch. Eventually implement Armijio for linesearch
        self._alpha_scaling = alpha_scaling
        
        # Set up visualization
        self._visualizer = visualizer
        self._logger = logger
        
    def run(self):
        iteration = 0
        
        while not self._is_converged():
            # Update last operating point and compute current one
            xs, us, costs = self._compute_operating_point()
            self._last_operating_point = self._current_operating_point
            self._current_operating_point = (xs, us, costs)
            
            # Linearize about current operating point
            As = []
            Bs = [[] for ii in range(self._num_players)]
            for k in range(self._horizon):
                A, B = self._dynamics.linearize_discrete(xs[k], [uis[k] for uis in us] )
                As.append(A)
                
                for ii in range(self._num_players):
                    Bs[ii].append(B[ii])
                    
            # Quadraticize the cost
            Qs = [[] for ii in range(self._num_players)]
        
        
        
        

In [ ]:
from sympy import symbols, Matrix, Function, simplify, exp, hessian

In [ ]:
x1, x2 = symbols('x1 x2')
f, g, h = symbols('f g h', cls=Function)

X = Matrix([x1,x2])
f = Matrix([-x1*x2*exp(-(x1**2 + x2**2)/2)])
h = 2*x1 + 3*x2
g =  x1**2 + x2**2 - 10

gradf = simplify(f.jacobian(X))
gradf

In [ ]:
import numpy as np
print(np.array([[0],[0],[0]]).shape)

In [ ]:
total_cost = torch.zeros(1, 1, requires_grad=True).double()

#for ii in range(self._num_players):
    xi_idx, yi_idx = self._position_indices[self._player_id]

    for jj in range(self._num_players):
        if self._player_id == jj:
            continue

        # Compute relative distance.
        xj_idx, yj_idx = self._position_indices[jj]
        dx = x[xi_idx, 0] - x[xj_idx, 0]
        dy = x[yi_idx, 0] - x[yj_idx, 0]
        relative_distance = torch.sqrt(dx*dx + dy*dy)

        total_cost += min(
            relative_distance - self._max_distance, 0.0)**2

return total_cost

In [ ]:
# This is me coding up part of the reach-avoid problem
for k in range(5):
    for j in range(k):
        print(j)
        

for k in range(self._horizon):
    l_k = dist(target_i, xs[k])
    for j in range(k):
        g_j1 = dist(xs_i[k], obstacle)
        g_j2 = dist(xs_i[k], xs_j[k])
        #g_j3 = dist(xs_i[k], curb)
        

In [ ]:
            k_tracker = 0
            eps = 0.01
            for k in range(self._horizon):
                #print("xs is: ", xs)
                #print("xs[k][0:2] is: ", xs[k][0:2])
                #print("xs[k][5:7] is: ", xs[k][5:7])
                hold_new = np.linalg.norm(xs[k][0:2] - xs[k][5:7]) + 0.5 * eps * np.linalg.norm(us[0][k])**2
    
                if k == 0:
                    hold = hold_new #This is the first computed distance (at t=0)
                elif hold_new < hold:
                    hold = hold_new
                    k_tracker = k